In [ ]:
import os
import sys
from tqdm import tqdm
import re
from itertools import combinations
import pandas as pd
import numpy as np

Read the results of the optimization. As we did run the optimization with multiple seeds, we need to merge these results by adhering to epsilon dominance among them. First, let's read all different csvs:

In [2]:
# Get the list of optimization results files:

d_vars = dict()
objectives = dict()

for file in os.listdir("../outputs/"):
    if "baseline_opt_dvs" in file:
        seed_search = re.search("seed_(.+?).csv", file)
        seed = int(seed_search.group(1))
        df = pd.read_csv(f"../outputs/{file}").rename(
            columns={"Unnamed: 0": "old_index"}
        )
        df["seed_index"] = seed
        df = df.set_index(["seed_index", "old_index"])
        d_vars[seed] = df

    elif "baseline_opt_objs" in file:
        seed_search = re.search("seed_(.+?).csv", file)
        seed = int(seed_search.group(1))
        df = pd.read_csv(f"../outputs/{file}").rename(
            columns={"Unnamed: 0": "old_index"}
        )
        df["seed_index"] = seed
        df = df.set_index(["seed_index", "old_index"])
        objectives[seed] = df

all_seeds = list(objectives.keys())

# .drop(columns=["Unnamed: 0"])

Now, the roadmap for checking for epsilon dominance is:
1) Merge all dataframes to get a single long one. Use hieararchical indexing [(seed, old_index)]
2) Divide every solution's objective values by epsilons and get the floor. Record these values side-by-side to the same dataframe as new columns
3) Generate an empty list of dominated indices
4) Go through all two-way combination of indices in our dataframe. Check them, if one is dominated, eliminate it (put in the dominated list)
5) If not, check if their floors are exactly the same (in the same epsilon box). If not, continue. If yes, keep the one that is closest to the ideal corner and eliminate the other one.



In [3]:
full_dvs = pd.concat(d_vars.values())
full_objectives = pd.concat(objectives.values())
full_objectives

Egypt_irr_def  HAD_min_level  Sudan_irr_def  \
seed_index old_index                                                
10         0               3.169425     178.101013       0.000000   
           1               4.394575     177.009079       0.000000   
           2               0.890999     174.851746       0.000000   
           3               1.268295     175.973740       0.000000   
           4               2.951919     178.100632       0.000000   
...                             ...            ...            ...   
40         292             0.672353     175.734055       0.000065   
           293             0.986166     176.575470       0.000000   
           294             0.189866     172.295990       0.000000   
           295             3.274011     178.045959       0.000000   
           296             0.099643     169.020828       0.000000   

                      Ethiopia_hydroenergy  
seed_index old_index                        
10         0                     13.520777  
           1                     14.200665  
           2                     13.913525  
           3                     13.901584  
           4                     13.404197  
...                                    ...  
40         292                   13.601729  
           293                   13.623336  
           294                   13.634564  
           295                   13.807129  
           296                   13.424238  

[1469 rows x 4 columns]

In [4]:
full_dvs

v0        v1        v2        v3        v4  \
seed_index old_index                                                     
10         0          0.060325  0.877317  0.561054  0.119365  0.000000   
           1          0.055795  0.237636  0.441866  0.088090  0.065339   
           2          0.058701  0.707590  0.275926  0.124304  0.095253   
           3          0.055258  0.189708  0.289355  0.114021  0.099022   
           4          0.056138  0.760261  0.670834  0.103302  0.027076   
...                        ...       ...       ...       ...       ...   
30         269        0.056300  0.723676  0.102094  0.113643  0.091694   
           270        0.056875  0.540134  0.223925  0.123836  0.099220   
           271        0.054975  0.648628  0.135663  0.115883  0.051150   
           272        0.056385  0.575414  0.127654  0.115965  0.092790   
           273        0.056160  0.550720  0.427092  0.124318  0.090522   

                            v5        v6        v7        v8        v9  ...  \
seed_index old_index                                                    ...   
10         0          1.000000  1.000000  0.701520  0.698278 -0.012731  ...   
           1          0.907105  0.248029  0.578255  0.755401 -0.254503  ...   
           2          0.793470  0.956274  0.696490  0.701188  0.102418  ...   
           3          0.868064  0.460202  0.325692  0.653089  0.684467  ...   
           4          0.899806  0.946472  0.664383  0.778911  0.114547  ...   
...                        ...       ...       ...       ...       ...  ...   
30         269        0.532893  0.998140 -0.833476  0.029757 -0.013656  ...   
           270        0.991454  0.353574 -0.794127  0.133992 -0.187415  ...   
           271        0.858517  0.306839 -0.995017  0.365020 -0.110011  ...   
           272        0.462794  0.924607 -0.846450  0.012077 -0.150837  ...   
           273        0.296131  0.952517 -0.883028  0.063930 -0.046986  ...   

                          v195      v196      v197      v198      v199  \
seed_index old_index                                                     
10         0          0.438421  0.569075  0.062209  0.882537  0.264937   
           1          0.978921  0.618624  0.716298  0.551495  0.353163   
           2          0.469593  0.802860  0.286315  0.609391  0.600699   
           3          0.330390  0.698901  0.582098  0.371123  0.935251   
           4          0.571411  0.532694  0.120599  0.593334  0.323594   
...                        ...       ...       ...       ...       ...   
30         269        0.251732  0.298113  0.012818  0.249242  0.010624   
           270        0.037061  0.799346  0.805750  0.758451  0.428693   
           271        0.270202  0.474333  0.062147  0.683956  0.780931   
           272        0.233209  0.435533  0.177848  0.705503  0.106179   
           273        0.124401  0.586775  0.575918  0.755981  0.752343   

                          v200      v201      v202      v203      v204  
seed_index old_index                                                    
10         0          0.333443  0.528249  0.342959  0.984255  1.000000  
           1          0.970628  0.778533  0.246930  0.869173  0.567703  
           2          0.236794  0.673422  0.407574  0.400413  0.132143  
           3          0.776950  0.627426  0.588914  0.470923  0.459146  
           4          0.802325  0.816070  0.324663  0.585280  0.802974  
...                        ...       ...       ...       ...       ...  
30         269        0.610932  0.409974  0.654778  0.762248  0.552730  
           270        0.000000  0.754060  0.497425  0.332644  0.528915  
           271        0.452117  0.354425  0.463992  0.351346  0.554518  
           272        0.664910  0.394399  0.647579  0.711036  0.535331  
           273        0.507355  0.465115  0.495922  0.744878  0.568570  

[1469 rows x 205 columns]

In [5]:
epsilons = [0.1, 0.1, 0.1, 0.1]
directions = [1, -1, 1, -1]

for i, column in enumerate(full_objectives.columns):
    full_objectives[f"{column}_floor"] = full_objectives[column] // epsilons[i]

full_objectives.head()

Egypt_irr_def  HAD_min_level  Sudan_irr_def  \
seed_index old_index                                                
10         0               3.169425     178.101013            0.0   
           1               4.394575     177.009079            0.0   
           2               0.890999     174.851746            0.0   
           3               1.268295     175.973740            0.0   
           4               2.951919     178.100632            0.0   

                      Ethiopia_hydroenergy  Egypt_irr_def_floor  \
seed_index old_index                                              
10         0                     13.520777                 31.0   
           1                     14.200665                 43.0   
           2                     13.913525                  8.0   
           3                     13.901584                 12.0   
           4                     13.404197                 29.0   

                      HAD_min_level_floor  Sudan_irr_def_floor  \
seed_index old_index                                             
10         0                       1781.0                  0.0   
           1                       1770.0                  0.0   
           2                       1748.0                  0.0   
           3                       1759.0                  0.0   
           4                       1781.0                  0.0   

                      Ethiopia_hydroenergy_floor  
seed_index old_index                              
10         0                               135.0  
           1                               142.0  
           2                               139.0  
           3                               139.0  
           4                               134.0

In [6]:
def compare_two_solutions(index1, index2):
    # Returns the dominated index

    copy_df = full_objectives.copy()

    first_sub = copy_df.loc[index1][:4]
    second_sub = copy_df.loc[index2][:4]

    for i, direct in enumerate(directions):
        first_sub[i] *= direct
        second_sub[i] *= direct

    if all(first_sub < second_sub):
        return index2
    elif all(second_sub < first_sub):
        return index1
    else:
        return None


def epsilon_comparison(index1, index2, reference_point):
    copy_df = full_objectives.copy()

    first_sub = copy_df.loc[index1]
    second_sub = copy_df.loc[index2]

    if all(first_sub[4:] == second_sub[4:]):
        distance1 = np.linalg.norm(first_sub[:4] - reference_point)
        distance2 = np.linalg.norm(second_sub[:4] - reference_point)

        if distance1 < distance2:
            return index2
        elif distance2 < distance1:
            return index1

In [7]:
dominated_indices = []
reference_point = np.array([0, 185, 0, 20])

indices = list(full_objectives.index)
two_way_combs = combinations(indices, 2)
for comb in tqdm(two_way_combs):
    item1 = comb[0]
    item2 = comb[1]

    if (item1 in dominated_indices) or (item2 in dominated_indices):
        continue

    comparison = compare_two_solutions(item1, item2)

    if comparison:
        dominated_indices.append(comparison)
        continue

    epsilon_check = epsilon_comparison(item1, item2, reference_point)
    if epsilon_check:
        dominated_indices.append(epsilon_check)

1078246it [14:37, 1228.57it/s]


In [9]:
len(dominated_indices)

292

Now, we have all the dominated indices. After dropping these, we can generate two csv's: one for decision variables, one for objective values.

In [10]:
final_objs = full_objectives.loc[~full_objectives.index.isin(dominated_indices)]
# final_objs = final_objs.reset_index().drop(columns=["seed_index", "old_index"])
final_objs = final_objs.drop(columns=[i for i in final_objs.columns if "floor" in i])
final_dvs = full_dvs.loc[~full_dvs.index.isin(dominated_indices)]
# final_dvs = final_dvs.reset_index().drop(columns=["seed_index", "old_index"])

In [20]:
final_objs.sort_index().to_csv("merged_objectives.csv", index=False)
final_dvs.sort_index().to_csv("merged_dvs.csv", index=False)